# EJERCICIO DE PROCESAMIENTO Y MACHINE LEARNING
### USAC - CUNORI

In [ ]:
# Cargar librerias necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
# Cargar base de datos
file = 'https://raw.githubusercontent.com/oestradavargas/Diplomado_Cunori_2023/main/BD_frijol.csv'
df = pd.read_csv(file, encoding = 'ISO-8859-1')
df.head()

In [ ]:
df.shape

### Procesamiento

In [ ]:
# Eliminar columnas innecesarias
df = df.drop('ID_Evento', axis = 1)

In [ ]:
# Revisar el tipo de variable
df.info()

In [ ]:
# Conteo de datos faltantes
df.isna().sum()

In [ ]:
# Eliminar registros cque contienen datos faltantes
df = df.dropna()

In [ ]:
# Verificar los datos faltantes
df.isna().sum()

In [ ]:
# Descripcion estadistica (variables cuantitativas)
df.describe()

In [ ]:
# Transformacion de variables
df['RDT'] = df['RDT'] / 1000

### Análisis gráfico

In [ ]:
# Gráfico de cajas y bigotes o boxplot
sns.boxplot(data = df, y = 'Pendiente')
plt.title('PENDIENTE (Boxplot)')
plt.ylabel('Pendiente (%)')
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de barras para variables categóricas
sns.countplot(data = df, x= 'Estructura')
plt.title('ESTRUCTURA DEL SUELO (Gráfico de barras)')
plt.xlabel('Estructura')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

In [ ]:
# Histograma
sns.histplot(data = df, x= 'RDT', kde=True)
plt.title('DISTRIBUCION DEL RENDIMIENTO (Histograma)')
plt.xlabel('Rendimiento ton/ha)')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de dispersión
sns.scatterplot(data = df, x = 'TotN', y= 'RDT')
plt.title('CANTIDAD DE NITROGENO vs RENDIMIENTO (Dispersión)')
plt.xlabel('N (kg/ha)')
plt.ylabel('Rendimiento (ton/ha)')
plt.tight_layout()
plt.show()

### Análisis de Machine Learning

In [ ]:
# Codificación de variables categóricas
df = pd.get_dummies(df, drop_first=True, dtype=int)

In [ ]:
df.head()

In [ ]:
# Separación de datos en entrenamiento y prueba (70% y 30%, respectivamente)
X = df.drop('RDT', axis=1)
y = df['RDT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

In [ ]:
# Creación del modelo
modelo = RandomForestRegressor(random_state=1, n_jobs=-1)
modelo.fit(X_train, y_train)

In [ ]:
# Evaluación del desempeño del modelo
predicciones = modelo.predict(X_test)
train_score = modelo.score(X_train, y_train) * 100
test_score = modelo.score(X_test, y_test) * 100
rmse = mean_squared_error(y_test, predicciones, squared=False)

print('\nDesempeño del modelo:')
print(f'\nTrain score: {train_score:.2f}%')
print(f'Test score: {test_score:.2f}%')
print(f'RMSE: {rmse:.2f} ton\n')

In [ ]:
# Relevancia de características
nombres_var = modelo.feature_names_in_
data_model = pd.concat([pd.Series(nombres_var), pd.Series(modelo.feature_importances_ )], axis=1)
data_model.columns = ['Feature', 'Values']
data_model = data_model.sort_values('Values', ascending=False)
data_model = data_model.iloc[0:5]

# Grafico
sns.barplot(data=data_model, x='Feature', y='Values', color = '#1f77b4')
plt.title('Relevancia de Variables (Random Forest: ' + str(round(test_score, 2)) + '%)')
plt.xticks(rotation=90)
plt.xlabel('Variable')
plt.ylabel('Relevancia media')
plt.tight_layout()
plt.show()

In [ ]:
# Dependencias parciales
for i in data_model['Feature']:
    PartialDependenceDisplay.from_estimator(modelo, X_train, [i])
    plt.title(i + ' (Dependencia parcial)')
    plt.xlabel(i)
    plt.ylabel('Dependencia parcial')
    plt.tight_layout()
    plt.show()
    plt.close()